#### Draw time series

In [ ]:
# Imports
import os
import numpy as np
import nibabel as nb
import pandas as pd
import scipy
import warnings
warnings.filterwarnings('ignore')

# Figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.express as px
from plot_utils import plotly_template

# pRFpy
from prfpy.stimulus import PRFStimulus2D
from prfpy.model import Iso2DGaussianModel
from prfpy.fit import Iso2DGaussianFitter


# Define folders
base_dir = '/home/mszinte/disks/meso_S/data/gaze_prf'
bids_dir = "{}".format(base_dir)
pp_dir = "{}/derivatives/pp_data".format(base_dir)
vdm_dir = "{}/derivatives/visual_dm".format(base_dir)

# settings
TR = 1.3
screen_distance_cm = 225
screen_size_cm = 69.8
subject2plot = 'sub-003'
roi2plot = 'V1'
cortical_mask = 'cortical'
fit_dir = '{}/{}/prf/fit'.format(pp_dir, subject2plot)
func_avg_dir = '{}/{}/func_avg'.format(pp_dir, subject2plot)
mask_dir = '{}/{}/masks'.format(pp_dir, subject2plot)
tsv_dir = '{}/{}/prf/tsv'.format(pp_dir, subject2plot)

# General figure settings
template_specs = dict(  axes_color="rgba(0, 0, 0, 1)",
                        axes_width=2,
                        axes_font_size=13,
                        bg_col="rgba(255, 255, 255, 1)",
                        font='Arial',
                        title_font_size=15,
                        plot_width=1.5)
fig_template = plotly_template(template_specs)

In [ ]:
# Load time series
fs_fn = '{}/{}_task-FullScreen_fmriprep_dct_avg.nii.gz'.format(func_avg_dir,subject2plot)
fs_ts = nb.load(fs_fn).get_fdata()

# Load roi mask
lh_mat = nb.load("{}/{}_{}_L.nii.gz".format(mask_dir, roi2plot, cortical_mask)).get_fdata()
rh_mat = nb.load("{}/{}_{}_R.nii.gz".format(mask_dir, roi2plot, cortical_mask)).get_fdata()
roi_mat = lh_mat + rh_mat
roi_mat[roi_mat==0] = np.nan

# Load pRF threshold masks
th_mat = nb.load('{}/{}_task-FullScreen_prf_threshold.nii.gz'.format(mask_dir,subject2plot)).get_fdata()

# Combined masks
th_roi_mat = th_mat*roi_mat

# Load fit parameters
fs_r2_mat = nb.load('{}/{}_task-FullScreen_par-r2.nii.gz'.format(fit_dir,subject2plot)).get_fdata()
fs_ecc_mat = nb.load('{}/{}_task-FullScreen_par-ecc.nii.gz'.format(fit_dir,subject2plot)).get_fdata()
fs_sd_mat = nb.load('{}/{}_task-FullScreen_par-sd.nii.gz'.format(fit_dir,subject2plot)).get_fdata()
fs_x_mat = nb.load('{}/{}_task-FullScreen_par-x.nii.gz'.format(fit_dir,subject2plot)).get_fdata()
fs_y_mat = nb.load('{}/{}_task-FullScreen_par-y.nii.gz'.format(fit_dir,subject2plot)).get_fdata()
fs_amp_mat = nb.load('{}/{}_task-FullScreen_par-amplitude.nii.gz'.format(fit_dir,subject2plot)).get_fdata()
fs_bsl_mat = nb.load('{}/{}_task-FullScreen_par-baseline.nii.gz'.format(fit_dir,subject2plot)).get_fdata()

In [ ]:
# create dataframe with masked data
fs_ts_df = pd.DataFrame({'r2': fs_r2_mat[th_roi_mat == True],
                         'ecc': fs_ecc_mat[th_roi_mat == True],
                         'sd': fs_sd_mat[th_roi_mat == True],
                         'x': fs_x_mat[th_roi_mat == True],
                         'y': fs_y_mat[th_roi_mat == True],
                         'amplitude': fs_amp_mat[th_roi_mat == True],
                         'baseline': fs_bsl_mat[th_roi_mat == True],
                         'time': [np.round(np.arange(1,fs_ts.shape[3]+1)*TR,decimals=2)]*np.sum(th_roi_mat == True),
                         'data_ts': fs_ts[th_roi_mat == True,:].tolist()})

# Get model timeseries
visual_dm_file = scipy.io.loadmat("{}/FullScreen_vd.mat".format(vdm_dir))
visual_dm = visual_dm_file['stim'].transpose([1,0,2])
stimulus = PRFStimulus2D(screen_size_cm=screen_size_cm, screen_distance_cm=screen_distance_cm, 
                         design_matrix=visual_dm, TR=TR)
gauss_model = Iso2DGaussianModel(stimulus=stimulus, normalize_RFs=True)
model_ts = np.zeros((fs_ts_df.index.stop,visual_dm.shape[2]))
for vox in fs_ts_df.index :
    model_ts[vox,:] = gauss_model.return_prediction(mu_x=fs_ts_df.x[vox], mu_y=fs_ts_df.y[vox], size=fs_ts_df.sd[vox],  
                                                    beta=fs_ts_df.amplitude[vox], baseline=fs_ts_df.baseline[vox])

fs_ts_df['model_ts'] = model_ts.tolist()

In [ ]:
# select voxels
roi_df = fs_ts_df.loc[(fs_ts_df.r2>0.6) & (fs_ts_df.sd<1.5) & (fs_ts_df.x<-1.5)].reset_index()
num_vox = 3

# get RF
gauss_fitter = Iso2DGaussianFitter(data=model_ts, model=gauss_model)
gauss_fitter.grid_fit(ecc_grid=roi_df.ecc[num_vox], 
                      polar_grid=np.angle(roi_df.x[num_vox] + 1j * roi_df.y[num_vox]),
                      size_grid=roi_df.sd[num_vox])
gauss_model.create_rfs()
rf = gauss_model.grid_rfs[0,:,:]
rf_norm = ((rf - np.min(rf)) / (np.max(rf) - np.min(rf)))*255

In [ ]:
# Subplot settings
rows, cols = 2, 2
margin_t, margin_b, margin_l, margin_r = 50, 50 ,50 ,50
fig_ratio = 5
fig_height = 1080/fig_ratio + (1080/fig_ratio*0.1) + margin_t+margin_b
fig_width = 1920/fig_ratio + 1920/fig_ratio + margin_l+margin_r
column_widths,row_heights = [1,1],[0.15,1]
sb_specs = [[{},{}],[{},{}]]
hover_data = 'Time: %{x:1.2f} s<br>' + 'z-score: %{y:1.2f}'
hover_model = 'Time: %{x:1.2f} s<br>' + 'z-score: %{y:1.2f}'

xaxis_range = [0,195]
yaxis_range = [-1,2]
yaxis_dtick = 1
x_tickvals = np.linspace(0,150,6)*TR
lwd_mot = np.array([ 32*TR,  20*TR, 0.5,  0.5])
dwd_mot = np.array([ 90*TR, 102*TR, 0.5,  0.5])
rwd_mot = np.array([ 61*TR,  61*TR, 0.85, 0.15])
uwd_mot = np.array([131*TR, 131*TR, 0.15, 0.85])
x0_all = np.array([0,10,42,52,70,80,112,122,140])*TR
x1_all = np.array([10,42,52,70,80,112,122,140,150])*TR
rolling = 3
data_col = 'rgba(0, 0, 0, 1)'
model_col = 'rgba(200, 0, 0, 1)'
subplot_titles = ['<b>{} timeseries </b> ({})'.format(roi2plot,subject2plot),'','','']
prf_xrange = [-8.9,8.9]
prf_yrange = [-5,5]
x_par_txt = 2.4
y_par_text = 4

# create figure
fig = make_subplots(rows=rows, cols=cols, specs=sb_specs, print_grid=False, vertical_spacing=0.05, horizontal_spacing=0.05, 
                    column_widths=column_widths, row_heights=row_heights,  subplot_titles=subplot_titles)

# Timeseries stim
for x0,x1 in zip(x0_all,x1_all):
    fig.add_shape(type='rect', xref='x', yref='y', x0=x0, y0=0, x1=x1, y1=1, 
                  line_width=2, fillcolor='black', line_color='white')
for [coord_tp] in zip([lwd_mot, dwd_mot, rwd_mot, uwd_mot]):
    fig.add_annotation(ax=coord_tp[0], x=coord_tp[1], ay=coord_tp[2], y=coord_tp[3], 
                       xref='x', yref='y', axref='x',ayref='y',
                       text='', showarrow=True, arrowhead=2, arrowcolor='white')

# time series data
fig.append_trace(go.Scatter(x=np.array(pd.Series(roi_df.time[num_vox]).rolling(window=rolling).mean())[::rolling],
                            y=np.array(pd.Series(roi_df.data_ts[num_vox]).rolling(window=rolling).mean())[::rolling],
                            name='<i>data<i>',
                            showlegend=True, mode='markers', marker_color=data_col, 
                            hovertemplate=hover_data,
                            line_width=0, opacity=1, marker_size=6),row=2, col=1)
                       
fig.append_trace(go.Scatter(x=roi_df.time[num_vox],
                            y=roi_df.model_ts[num_vox],
                            name='<i>model<i>',
                            showlegend=True, mode='lines', line_color=data_col, 
                            hovertemplate=hover_model,
                            line_width=2, opacity=1),row=2, col=1)

# pRF
fig.append_trace(go.Heatmap(x=np.linspace(prf_xrange[0], prf_xrange[1], visual_dm.shape[0]), 
                            y=np.linspace(prf_xrange[1], prf_xrange[0], visual_dm.shape[1]), z=rf_norm,
                            colorscale='viridis', showscale=False, hoverinfo='none',
                            ),row=2,col=2)

fig.add_annotation(x=roi_df.x[num_vox], ax=roi_df.x[num_vox], y=prf_yrange[0], ay=prf_yrange[0]-0.5,
                   xref='x4', yref='y4', axref='x4',ayref='y4', yanchor="top", showarrow=True,
                   text='<i>pRFx</i> = {:1.2g}°'.format(roi_df.x[num_vox]), arrowhead=2, arrowwidth=2.5)

fig.add_annotation(x=prf_xrange[1], ax=prf_xrange[1]+0.5, y=roi_df.y[num_vox], ay=roi_df.y[num_vox],
                   xref='x4', yref='y4', axref='x4',ayref='y4', yanchor="top", showarrow=True, 
                   text='<i>pRFy</i> = {:1.2g}°'.format(roi_df.y[num_vox]), textangle=-90, arrowhead=2, arrowwidth=2.5)
fig.add_shape(type='line', xref='x4', yref='y4', x0=prf_xrange[0], x1=prf_xrange[1], y0=roi_df.y[num_vox], y1=roi_df.y[num_vox], 
                  line_width=2, line_color='white', line_dash='dot')
fig.add_shape(type='line', xref='x4', yref='y4', x0=roi_df.x[num_vox], x1=roi_df.x[num_vox], y0=prf_yrange[0], y1=prf_yrange[1], 
                  line_width=2, line_color='white', line_dash='dot')

fig.add_annotation(x=x_par_txt, y=y_par_text, xref='x4', yref='y4', xanchor="left", font_color='white', showarrow=False,
                   text='<i>pRF R2</i> = {:1.2g}'.format(roi_df.r2[num_vox]))
fig.add_annotation(x=x_par_txt, y=y_par_text-1, xref='x4', yref='y4', xanchor="left", font_color='white', showarrow=False,
                   text='<i>pRF size</i> = {:1.2g}°'.format(roi_df.sd[num_vox]))
fig.add_annotation(x=x_par_txt, y=y_par_text-2, xref='x4', yref='y4',  xanchor="left", font_color='white', showarrow=False, 
                   text='<i>pRF ecc</i> = {:1.2g}°'.format(roi_df.ecc[num_vox]))
fig.add_annotation(x=x_par_txt, y=y_par_text-3, xref='x4', yref='y4',  xanchor="left", font_color='white', showarrow=False, 
                   text='<i>pRF angle</i> = {:3.0f}°'.format(np.angle(roi_df.x[num_vox] + 1j * roi_df.y[num_vox],deg=True)))

# set axis
for row in np.arange(rows):
    for col in np.arange(cols):
        fig.update_xaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1)
        fig.update_yaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1)
        
fig.layout.update(xaxis_range=xaxis_range, xaxis_title='', 
                  xaxis_visible=False, yaxis_visible=False,
                  yaxis_range=[0,1], yaxis_title='',
                  xaxis4_range=prf_xrange, xaxis4_title='', 
                  yaxis4_range=prf_yrange, yaxis4_title='', 
                  xaxis4_visible=False, yaxis4_visible=False,
                  xaxis3_tickvals=x_tickvals, xaxis3_ticktext=np.round(x_tickvals),
                  xaxis3_range=xaxis_range, xaxis3_title='Time (seconds)',
                  yaxis3_range=yaxis_range, yaxis3_title='z-score',yaxis3_dtick=yaxis_dtick,
                  template=fig_template, width=fig_width, height=fig_height, 
                  margin_l=margin_l+10, margin_r=margin_r-10, margin_t=margin_t-20, margin_b=margin_b+20,
                  legend_yanchor='top', legend_y=0.85, legend_xanchor='left', 
                  legend_x=0.02, legend_bgcolor='rgba(255,255,255,0)')

# show and save figure
fig.show(config={"displayModeBar": False})
fig.write_image("{}/{}_{}-timeseries.pdf".format(tsv_dir, subject2plot, roi2plot))
fig.write_html("{}/{}_{}-timeseries.html".format(tsv_dir, subject2plot, roi2plot),config={"displayModeBar": False})